In [1]:
from lusidtools.jupyter_tools import toggle_code

"""
Save externally calculated metrics and use them within the Valuation Engine

Futures
TimeVariant Properties
Transactions
Holdings
----------
"""

toggle_code("Toggle Docstring")

# Initial Margin using Time Variant Properties

In this notebook, we will demonstrate how you can process Futures Initial Margin. We will start with setting Initial Margin static data as Instrument time variant property 
then booking Initial Margin requirement on transaction upsert and finally applying daily adjustment on Initial Margin Cash Account.


## Table of Contents
* [1. Create Portfolio](#1.-Create-Portfolio)
* [2. Create Properties](#2.-Create-Properties)
* [3. Create Future Instrument](#3.-Create-Future-Instrument)
* [4. Update Initial Margin on Future Instrument](#4.-Update-Initial-Margin-on-Future-Instrument)
* [5. Create Transaction Types](#5.-Create-Transaction-Types)
* [6. Load Transaction Data](#6.-Load-Transaction-Data)
* [7. Daily Initial Margin Adjustment](#7.-Daily-Initial-Margin-Adjustment)

In [2]:
# Use first block to import generic non-LUSID packages
import io
import os
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
import json
import pytz
from IPython.core.display import HTML
from datetime import datetime

# Then import the key modules from the LUSID package (i.e. The LUSID SDK)
import lusid as lu
import lusid.api as la
import lusid.models as models

# And use absolute imports to import key functions from Lusid-Python-Tools and other helper package

from lusid.utilities import ApiClientFactory
from lusidjam import RefreshingToken
from lusidtools.cocoon.cocoon import load_from_data_frame
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame
from lusidtools.jupyter_tools import StopExecution
from lusidtools.cocoon.cocoon_printer import (
    format_instruments_response,
    format_portfolios_response,
    format_transactions_response,
)

# Set DataFrame display formats
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.options.display.float_format = "{:.2f}".format
pd.set_option('display.float_format', lambda x: '{:,.2f}'.format(x))
display(HTML("<style>.container { width:90% !important; }</style>"))

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

if secrets_path is None:
    secrets_path = os.path.join(os.path.dirname(os.getcwd()), "secrets.json")

api_factory = ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename=secrets_path,
    app_name="LusidJupyterNotebook",
)

api_status = pd.DataFrame(
    api_factory.build(lu.ApplicationMetadataApi).get_lusid_versions().to_dict()
)

display(api_status)

,api_version,build_version,excel_version,links
0,v0,0.6.11310.0,0.5.3243,"{'relation': 'RequestLogs', 'href': 'http://bo..."


In [3]:
holdings_api = api_factory.build(lu.TransactionPortfoliosApi)
instruments_api = api_factory.build(lu.InstrumentsApi)
aggregation_api = api_factory.build(lu.AggregationApi)
configuration_recipe_api = api_factory.build(lu.ConfigurationRecipeApi)
property_definitions_api = api_factory.build(lu.PropertyDefinitionsApi)
transaction_portfolio_api = api_factory.build(lu.api.TransactionPortfoliosApi)
system_configuration_api = api_factory.build(lu.api.SystemConfigurationApi)
transaction_configuration_api = api_factory.build(lu.api.TransactionConfigurationApi)

In [4]:
# Define scopes, codes and property keys
scope = "IMExampleNotebook"
portfolio_code = "FutureWithInitialMargin"
recipe_code = "futuresValuation"
base_currency = "EUR"
sub_holding_key = f"Transaction/{scope}/CashType"
fut_initial_margin_key = f"Instrument/{scope}/InitialMargin"
txn_initial_margin_key = f"Transaction/{scope}/InitialMargin"

# 1. Create Portfolio

In [5]:
try:
    transaction_portfolio_api.create_portfolio(
        scope=scope,
        create_transaction_portfolio_request=models.CreateTransactionPortfolioRequest(
            display_name=portfolio_code,
            code=portfolio_code,
            base_currency=base_currency,
            created="2010-01-01",
            sub_holding_keys=[sub_holding_key],
            instrument_scopes=[scope]
        ),
    )

except lu.ApiException as e:
    if(e.status == 401):
        print(e.reason)
    else:
        print(json.loads(e.body)["title"])

Could not create a portfolio with id 'FutureWithInitialMargin' because it already exists in scope 'IMExampleNotebook'.


# 2. Create Properties

In [6]:
properties = [
    ("CashType", "string", "Transaction","Perpetual"),
    ("InitialMargin", "number", "Transaction","Perpetual"),
    ("InitialMargin", "number", "Instrument","TimeVariant")
]

for property_code, dtype, domain, life_time in properties:
    try:
        property_definitions_api.create_property_definition(
            create_property_definition_request=models.CreatePropertyDefinitionRequest(
                domain=domain,
                scope=scope,
                code=property_code,
                display_name=property_code,
                data_type_id=models.ResourceId(code=dtype, scope="system"),
                life_time = life_time
            )
        )
    except lu.ApiException as e:
        print(json.loads(e.body)["title"])

Error creating Property Definition 'Transaction/IMExampleNotebook/CashType' because it already exists.
Error creating Property Definition 'Transaction/IMExampleNotebook/InitialMargin' because it already exists.
Error creating Property Definition 'Instrument/IMExampleNotebook/InitialMargin' because it already exists.


# 3. Create Future Instrument

In [7]:
# Define function that creates futures
def create_futures_contract(
        dom_ccy,
        contract_code,
        contract_month,
        contract_size,
        convention,
        country_id,
        fut_name,
        exchange_code,
        exchange_name,
        ticker_step,
        unit_value,
        ref_spot_price,
        start_date,
        maturity_date,
        fut_identifier,
        identifier_type
):
    ctc = models.FuturesContractDetails(
        dom_ccy=dom_ccy,
        contract_code=contract_code,
        contract_month=contract_month,
        contract_size=contract_size,
        convention=convention,
        country=country_id,
        description=fut_name,
        exchange_code=exchange_code,
        exchange_name=exchange_name,
        ticker_step=ticker_step,
        unit_value=unit_value,
    )
    futuredef = models.Future(
        start_date=start_date,
        maturity_date=maturity_date,
        identifiers={},
        contract_details=ctc,
        contracts=1,
        ref_spot_price=ref_spot_price,
        underlying=models.ExoticInstrument(
            instrument_format=models.InstrumentDefinitionFormat(
                "custom", "custom", "0.0.0"
            ),
            content="{}",
            instrument_type="ExoticInstrument",
        ),
        instrument_type="Future",
    )
    # persist the instrument
    futureDefinition = models.InstrumentDefinition(
        name=fut_name,
        identifiers={identifier_type: models.InstrumentIdValue(fut_identifier)},
        definition=futuredef,
    )
    batchUpsertRequest = {fut_identifier: futureDefinition}
    upsertResponse = instruments_api.upsert_instruments(request_body=batchUpsertRequest, scope=scope)
    futLuid = upsertResponse.values[fut_identifier].lusid_instrument_id
    print(futLuid)

In [8]:
start_date = datetime(2021, 6, 8, tzinfo=pytz.utc)
maturity_date = datetime(2022, 3, 8, tzinfo=pytz.utc)
dom_ccy = "EUR"
contract_code = "FGBL" #bbg=OE
contract_month = "H"
contract_size = 100000
convention = "ActualActual"
country_id= "DE"
fut_name = "EURO-BUND FUTURE Mar22"
exchange_code = "EUREX"
exchange_name ="Eurex"
ticker_step = 0.01
unit_value = 10
ref_spot_price_val = 0
identifier = "FutBund002"
identifier_type = "ClientInternal"

# Create Futures Contract function
create_futures_contract(
    dom_ccy,
    contract_code,
    contract_month,
    contract_size,
    convention,
    country_id,
    fut_name,
    exchange_code,
    exchange_name,
    ticker_step,
    unit_value,
    ref_spot_price_val,
    start_date,
    maturity_date,
    identifier,
    identifier_type
)

LUID_0000S7ZY


# 4. Update Initial Margin on Future Instrument
Initial Margin is associated to Future Contract definition using time variant Instrument Properties.

In [9]:
# Define function that updates futures initial margin
def upsert_initial_margin_property(
    fut_identifier,
    fut_identifier_type,
    margin_value,
    effective_from):
    property_key = fut_initial_margin_key
    property_request = models.UpsertInstrumentPropertyRequest(
        identifier = fut_identifier,
        identifier_type =  fut_identifier_type,
        properties =[
                models.ModelProperty(
                    key=property_key,
                    value=models.PropertyValue(
                        metric_value=models.MetricValue(
                            value=margin_value
                        )
                    ),
                    effective_from=effective_from
                )
            ])
    upsert_response = instruments_api.upsert_instruments_properties(scope = scope, 
                        upsert_instrument_property_request = [property_request])
    print(f"{property_key} upserted at {upsert_response.as_at_date}.")

In [10]:
# Read in initial margin data from file
futures_initial_margin_df = pd.read_csv("data/fut_initial_margin.csv")
futures_initial_margin_df.head()

,fut_identifier,intial_margin,effective_date
0,FutBund002,"4,525.25",2022-02-14T08:00:00Z
1,FutBund002,"4,527.00",2022-02-17T08:00:00Z
2,FutBund002,"4,523.00",2022-02-21T08:00:00Z


In [11]:
for _ , row in futures_initial_margin_df.iterrows():
    upsert_initial_margin_property(row["fut_identifier"], 
                                   identifier_type,
                                   row["intial_margin"],
                                   row["effective_date"])

Instrument/IMExampleNotebook/InitialMargin upserted at 2023-05-16 16:28:11.464618+00:00.
Instrument/IMExampleNotebook/InitialMargin upserted at 2023-05-16 16:28:11.635033+00:00.
Instrument/IMExampleNotebook/InitialMargin upserted at 2023-05-16 16:28:11.792163+00:00.


# 5. Create Transaction Types

In [12]:
# New Side for Initial Margin Cash Movement
side_list = [
    ("IM-Side",
     models.SideDefinitionRequest(
         security="Txn:SettlementCurrency",
         currency="Txn:SettlementCurrency",
         rate="SettledToPortfolioRate",
         units=txn_initial_margin_key,
         amount=txn_initial_margin_key
     ))
]

In [13]:
# List of Transaction types
transaction_type_req = [
   ("OpenContract", models.TransactionTypeRequest(
        aliases=[
            models.TransactionTypeAlias(
                type="OpenContract",
                description="Open a long future contract",
                transaction_class="Futures",
                transaction_roles="LongLonger",
            )
        ],
        movements=[
            models.TransactionTypeMovement(
                movement_types="StockMovement",
                side="Side1",
                direction=1,
                properties=None,
                mappings=[],
                name="Open contract"
            ),
            models.TransactionTypeMovement(
                movement_types="CashCommitment",
                side="IM-Side",
                direction=1,
                properties=None,
               mappings=[
                    models.TransactionPropertyMappingRequest(
                        property_key=sub_holding_key, set_to="Futures Initial Margin")],
                name="Initial Margin"
            )
        ],
        properties=None,
    )),
   
   ("IMAdjustment", models.TransactionType(
        aliases=[
            models.TransactionTypeAlias(
                type="IMAdjustment",
                description="Adjust Initial Margin",
                transaction_class="Futures",
                transaction_roles="AllRoles",
            )
        ],
        movements=[
            models.TransactionTypeMovement(
                movement_types="StockMovement",
                side="Side1",
                direction=1,
                properties=None,
                mappings=[],
                name="Open contract"
            ),
            models.TransactionTypeMovement(
                movement_types="CashCommitment",
                side="IM-Side",
                direction=1,
                properties=None,
                 mappings=[
                    models.TransactionPropertyMappingRequest(
                        property_key=sub_holding_key, set_to="Futures Initial Margin")],
                name="IM Increase/Decrease"
            )
        ],
        properties=None,
    ))
]

current_sides = [
   side.side for side in system_configuration_api.list_configuration_transaction_types().side_definitions]

for (side, side_req) in side_list:

    if side in list(current_sides):

        print(f"{side} already exists in LUSID")

    else:

        response = transaction_configuration_api.set_side_definition(
            side=side, side_definition_request=side_req)

        print(f"Side {side} has been created in LUSID")

for(type, type_req) in transaction_type_req:
    transaction_configuration_api.set_transaction_type("default", type, type_req)
    print(f"{type} has been created in LUSID")

IM-Side already exists in LUSID
OpenContract has been created in LUSID
IMAdjustment has been created in LUSID


# 6. Load Transaction Data
Create a Future Position on FutBund002 by upserting a new transaction.

In [14]:
def upsert_transaction(txn_id, txn_type,effective_date,txn_units,price,total_consideration,im_value):
    transaction_requests=[
         models.TransactionRequest(
                    transaction_id=txn_id,
                    type=txn_type,
                    instrument_identifiers={ "Instrument/default/ClientInternal": identifier },
                    transaction_date=effective_date,
                    settlement_date=effective_date,
                    units=txn_units,
                    transaction_price=models.TransactionPrice(
                        price=price, type="Price"
                    ),
                    total_consideration=models.CurrencyAndAmount(
                        amount=total_consideration, currency=base_currency
                    ),
                    properties={txn_initial_margin_key: models.PerpetualProperty(
                        key=txn_initial_margin_key,
                    value=models.PropertyValue(metric_value=models.MetricValue(value=im_value)))}
                )
    ]

    transaction_portfolio_api.upsert_transactions(
            scope=scope,
            code=portfolio_code,
            transaction_request=transaction_requests)
    
# Open Future Position and compute Initial Margin requirement
txn_units = 10
effective_date = "2022-02-14T09:00:00Z"
# Get Initial Margin Property then compute the margin requirement
fut_initial_margin = instruments_api.get_instrument_properties(identifier_type = identifier_type, identifier = identifier, 
    effective_at = effective_date, scope= scope).properties.get(fut_initial_margin_key).value.metric_value.value

txn_initial_margin = fut_initial_margin * txn_units

upsert_transaction("txn_01","OpenContract",effective_date,txn_units,135,1350000,txn_initial_margin)

# 7. Daily Initial Margin Adjustment

Here we are looking at the daily adjustment needed to keep the Initial Margin Cash Account up to date. First, we compute the margin change between T and T-1 using the formula:  $Units * (IM_{T} - IM_{T-1})$.
Then we book the margin adjustment by upserting a new transaction with transaction type **IMAdjustment**.

In [15]:
def get_daily_fut_position(effective_date):
    columns_to_rename = {
            "instrument_uid": "LusidInstrumentId",
            "units":"Units",
            "cost.amount":"Cost",
            "currency":"Currency",
            f"CashType({scope}-SubHoldingKeys)": "CashType"
        }
    columns_to_drop = ["instrument_scope","instrument_scope","SourcePortfolioScope(default-Properties)","SourcePortfolioId(default-Properties)","holding_type",
                       "settled_units","cost.currency","cost_portfolio_ccy.amount","cost_portfolio_ccy.currency","holding_type_name"]

    holdings = lusid_response_to_data_frame(transaction_portfolio_api.get_holdings(scope,portfolio_code,effective_at = effective_date),rename_properties=True)
    holdings = holdings.drop(columns = columns_to_drop)
    result=holdings[columns_to_rename.keys()]
    result.rename(columns=columns_to_rename, inplace=True)
    return result

## Day 1

On 2022-02-16T09:00:00Z, no Initial Margin change was recorded, no adjustment needed.

In [16]:
effective_date_1 = "2022-02-16T09:00:00Z"
get_daily_fut_position(effective_date_1)

,LusidInstrumentId,Units,Cost,Currency,CashType
0,LUID_0000S7ZY,10.00,"1,350,000.00",EUR,<Not Classified>
1,CCY_EUR,"45,252.50","45,252.50",EUR,Futures Initial Margin


In [17]:
def compute_initial_margin_adjustment(date_1, date_2):
    
    # Get Future Position on date_2
    fut_units = transaction_portfolio_api.get_holdings(scope,portfolio_code,effective_at = date_2).values[0].units
    # IM on date_1
    fut_initial_margin_1 = instruments_api.get_instrument_properties(identifier_type = identifier_type, identifier = identifier, 
        effective_at = date_1, scope= scope).properties.get(fut_initial_margin_key).value.metric_value.value
    # IM on date_2
    fut_initial_margin_2 = instruments_api.get_instrument_properties(identifier_type = identifier_type, identifier = identifier, 
        effective_at = date_2, scope= scope).properties.get(fut_initial_margin_key).value.metric_value.value

    return fut_units * (fut_initial_margin_2 - fut_initial_margin_1)
    

## Day 2

On 2022-02-17T09:00:00Z, we get a new Initial Margin worth **4,527**. We compute the required adjustment and update IM account using CashType = 'Futures Initial Margin'

In [23]:
effective_date_2 = "2022-02-17T09:00:00Z"

adjustment_amount = compute_initial_margin_adjustment(effective_date_1, effective_date_2)
print(f"The required Initial Margin adjustment is {adjustment_amount} EUR")

# Book IM adjustment
upsert_transaction("IM_01","IMAdjustment",effective_date_2,0,0,0,adjustment_amount)


The required Initial Margin adjustment is 17.5 EUR


In [24]:
get_daily_fut_position(effective_date_2)

,LusidInstrumentId,Units,Cost,Currency,CashType
0,LUID_0000S7ZY,10.00,"1,350,000.00",EUR,<Not Classified>
1,CCY_EUR,"45,270.00","45,270.00",EUR,Futures Initial Margin


## Day 3

On 2022-02-21T09:00:00Z, we get a new Initial Margin worth **4,523**. We compute the required adjustment and update IM account using CashType = **'Futures Initial Margin'**

In [22]:
effective_date_3 = "2022-02-21T09:00:00Z"

adjustment_amount = compute_initial_margin_adjustment(effective_date_2, effective_date_3)
print(f"The required Initial Margin adjustment is {adjustment_amount} EUR")

# Book IM adjustment
upsert_transaction("IM_02","IMAdjustment",effective_date_3,0,0,0,adjustment_amount)

The required Initial Margin adjustment is -40.0 EUR


In [21]:
get_daily_fut_position(effective_date_3)

,LusidInstrumentId,Units,Cost,Currency,CashType
0,LUID_0000S7ZY,10.00,"1,350,000.00",EUR,<Not Classified>
1,CCY_EUR,"45,230.00","45,230.00",EUR,Futures Initial Margin
